<a href="https://colab.research.google.com/github/FernandoIGD12/UTEC_programming101/blob/main/Intro_AI/proyecto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [167]:
# Import the necessary libraries and function
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [103]:
# download the urbania dataset with property listing from all over Peru
!gdown --id 1d_M8c0wP8ID0jlq8WjTKAcCk2EsiX-ys

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1d_M8c0wP8ID0jlq8WjTKAcCk2EsiX-ys
From (redirected): https://drive.google.com/uc?id=1d_M8c0wP8ID0jlq8WjTKAcCk2EsiX-ys&confirm=t&uuid=5c48125c-00ad-4e80-8b4c-e19f3f575f8d
To: /content/urbania_2025-06-19.csv
100% 110M/110M [00:00<00:00, 195MB/s] 


In [255]:
# Load the dataset into a pandas dataframe
df = pd.read_csv('urbania_2025-06-19.csv', encoding = 'latin')
df.replace('?', np.nan, inplace = True)
df.head()

,website,code,property_type,property_subtype,operation_type,region_lv1,region_lv2,region_lv3,name,address,...,clinic_hospital_num,atm_num,bus_station_num,subway_station_num,marketplace_num,university_num,library_num,img_urls,description,attributes
0,Urbania,59971409,HOUSE,Casa,FOR_SALE,Lima,Lima,La Molina,Oportunidad por viaje en RincÃ³n de la Planicie,"RINCON DE LA PLANICIE, La Planicie, La Molina,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://img10.naventcdn.com/avisos/111/00/59/9...,ANTES: $ 549 000 AHORA: $ 499 000 CÃDIGO: CV....,Areas de BBQ () | Ãreas de lavanderÃ­a () | J...
1,Urbania,62177554,APARTMENT,Departamento,FOR_SALE,Lima,Lima,San Isidro,DE LUJO CON JARDÃN! DEPARTAMENTO EN VENTA ZON...,"Calle Chabrier, San Isidro, Lima, Lima",...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,https://img10.naventcdn.com/avisos/111/00/62/1...,Una joya!!! Es un DEPA-CASA en Exclusivo Edifi...,Ascensores (1) | JardÃ­nes () | Patios () | Te...
2,Urbania,60534780,APARTMENT,Departamento,FOR_SALE,Lima,Lima,Santiago de Surco,OCASION!! VENDO DEPARTAMENTO FULL EQUIPADO EDI...,Av CircunvalaciÃ³n del Golf 148 - PRECIO DE OC...,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,https://img10.naventcdn.com/avisos/111/00/60/5...,VENDO DEPARTAMENTO EN EL EDIFICIO MOON GOLF LO...,Areas de BBQ () | Ãreas de lavanderÃ­a () | A...
3,Urbania,62550452,HOUSE,Casa,FOR_SALE,Lima,Lima,Santiago de Surco,EXCLUSIVA RESIDENCIA - JR TOMASAL - SURCO,"JR TOMASAL 489 SANTIAGO DE SURCO, San Jorge, S...",...,2.0,NaN,NaN,1.0,NaN,NaN,NaN,https://img10.naventcdn.com/avisos/111/00/62/5...,EXCLUSIVA Y ACOGEDORA RESIDENCIA EN CALLE TOMA...,Areas de BBQ () | Ãreas de lavanderÃ­a () | J...
4,Urbania,62699953,APARTMENT,Departamento,FOR_SALE,Lima,Lima,Miraflores,VENTA DEPARTAMENTO VISTA AL MAR MIRAFLORES DUPLEX,"Av. MalecÃ³n Cisneros 1100, Malecon Reserva, M...",...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,https://img10.naventcdn.com/avisos/111/00/62/6...,Venta de DÃºplex de lujo finos acabados. estr...,Areas de BBQ () | Ãreas de lavanderÃ­a () | A...


After a preliminary inspection of the features, we note that some of them do not contribute at all to the task at hand so we delete them. For the case of price, we keep only the price in USD, as the local currency price ('price') is subject to changes in the exchange rate. Then, we filter only listings from the Lima metropolitan area and those that are on sale and not for rent.

In [256]:
# Drop the irrelevant columns
df.drop(['website', 'code', 'name', 'address', 'lat', 'lng', 'constructed_area_unit', 'total_area_unit', 'price', 'local_currency', 'exchange_rate', 'fetched_price_currency',
         'contact_name', 'contact_email', 'contact_phone', 'last_edited', 'url', 'img_urls', 'description', 'attributes'],axis = 1, inplace = True)

# Filter the desired regions and sale listings
df = df[(df['region_lv1'] == 'Lima') & (df['region_lv2'] == 'Lima') & (df['operation_type'] == 'FOR_SALE')]
df.drop(['region_lv1', 'region_lv2', 'operation_type'], axis = 1, inplace = True)

In [257]:
print(df.shape)
df.isnull().sum().sort_values(ascending = False)

(29597, 38)


,0
university_num,29070
subway_station_num,28686
elevator_num,28527
library_num,28146
marketplace_num,27294
fuel_station_num,26637
building_floors,26626
bus_station_num,25499
floor_num,24566
atm_num,22064


In [250]:
to_delete = ['building_floors', 'floor_num', 'elevator_num']
df.drop(to_delete, axis = 1, inplace = True)

to_replace_zeros = ['constructed_area', 'bedrooms_num', 'bathrooms_num', 'garage_num', 'price_per_constructed_sq_m_usd', 'constructed_sq_m_region_diff_pc', 'school_num', 'restaurant_num', 'kindergarten_num', 'place_of_worship_num', 'pharmacy_num', 'fuel_station_num', 'bank_num', 'cafe_bar_num', 'clinic_hospital_num', 'atm_num', 'bus_station_num', 'subway_station_num', 'marketplace_num', 'university_num', 'library_num']
df[to_replace_zeros] = df[to_replace_zeros].replace(np.nan, 0)

df.dropna(subset=['total_area', 'region_lv3'], inplace=True)


In [251]:
median_year = df['construction_year'].median()
df['construction_year'].fillna(median_year, inplace=True)

most_frequent_condition = df['condition'].value_counts().idxmax()
df['condition'].fillna(most_frequent_condition, inplace=True)

In [252]:
property_type_dummies = pd.get_dummies(df['property_type'])
property_subtype_dummies = pd.get_dummies(df['property_subtype'])
region_dummies = pd.get_dummies(df['region_lv3'])
df = pd.concat([df, property_type_dummies, property_subtype_dummies, region_dummies], axis=1)
df.drop(['property_type', 'property_subtype', 'region_lv3'], axis=1, inplace=True)